In [10]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
# from nltk import word_tokenize

In [11]:
import nltk
nltk.download("punkt_tab")
nltk.download("stopwords")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
tweets = pd.read_csv("./Tweets.csv")

In [ ]:
tweets.head()


In [ ]:
tweets.info()

In [ ]:
tweets.shape

In [ ]:
tweets.text.tail()

### Word Cloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS
# Building a word cloud for negative tweet text
negative_tweets = tweets["text"][tweets["airline_sentiment"] =="negative"]
joined_negative_tweets = "".join(negative_tweets)

my_stop_words = set(STOPWORDS)
# reg = r"@"|
my_stop_words = my_stop_words.union(["@VirginAmerica", "united","flight","AmericanAir","USAirways","SouthwestAir","JetBlue"])
word_cloud = WordCloud(stopwords=my_stop_words, regexp=r'\b[^\d\W][^\d\W]+\b').generate(joined_negative_tweets)

#show word cloud
plt.imshow(word_cloud, interpolation="bilinear")
plt.axis("off")




In [ ]:
from wordcloud import WordCloud, STOPWORDS
# Building a word cloud for negative tweet text
# positive_neutral_tweets = tweets["text"][~tweets["airline_sentiment"] =="negative"]
positive_neutral_tweets = tweets["text"][tweets["airline_sentiment"] !="negative"]
joined_positive_neutral_tweets = "".join(positive_neutral_tweets)

my_stop_words = set(STOPWORDS)
# reg = r"@"|
my_stop_words = my_stop_words.union(["@VirginAmerica", "united","flight","AmericanAir","USAirways","SouthwestAir", "JetBlue","VirginAmerica","co", "t"])
word_cloud = WordCloud(background_color="white", stopwords=my_stop_words, regexp=r'\b[^\d\W][^\d\W]+\b').generate(joined_positive_neutral_tweets)

#show word cloud
plt.imshow(word_cloud, interpolation="bilinear")
plt.axis("off")




### Data Preprocessing and Feature Engineering

In [13]:
tweets = tweets[["text","airline_sentiment"]]

In [14]:
#Encode the target label
from sklearn.preprocessing  import LabelEncoder
le = LabelEncoder()
tweets.airline_sentiment = le.fit_transform(tweets.airline_sentiment)

In [15]:
# Defie the text preprocessor function
def preprocessor(text):
    text = re.sub(r"@[d W]+", "", text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))

    return text


In [16]:
# Cleaning the text
tweets.text = tweets.text.apply(preprocessor)

In [ ]:
# skip this
# # Add text_length feature
# from nltk import word_tokenize
# tokens = [word_tokenize(word) for word in tweets.text]
# length =[]
# for i in range(len(tokens)):
#     length.append(len(tokens[i]))

# tweets["text_length"] = length

## Building the Classifier

In [17]:
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

#defining the stop words
stop_words = stopwords.words("english")

stop_words_2 = list(set(stop_words).union({"AmericanAir", "USAirways", "SouthwestAir", "JetBlue", "VirginAmerica"}))


def tokenizer(text):
    return word_tokenize(text)

def tokenizer_porter(text):
    return [PorterStemmer().stem(text) for text in word_tokenize(text)]

In [18]:
# Define X and y
y = tweets.airline_sentiment
X = tweets.text


In [19]:
from sklearn.model_selection import train_test_split
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=456, stratify=y)

#### Traing a Logistic Regression Model

In [ ]:
from  sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
# from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf = ColumnTransformer([("vectorizer", TfidfVectorizer(lowercase=False),0), ("nothing", "passthrough",[1])] )
tfidf = TfidfVectorizer(lowercase=False)


param_grid =[
    {
        "vect__ngram_range" : [(1,1),(1,2)],
        "vect__stop_words" : [None, stop_words],
        "vect__tokenizer" : [tokenizer, tokenizer_porter],
        "clf__penalty" : ["l2"],
        "clf__C" : [0.1, 10.0]
    },
    {
        "vect__ngram_range" : [(1,1),(1,2)],
        "vect__stop_words" : [stop_words, stop_words_2],
        "vect__tokenizer" : [tokenizer],
        "vect__use_idf" : [False],
        "vect__norm" : [None],
        "clf__penalty" : ["l2","l1"],
        "clf__C" : [1.0, 10.0]

    }
]

lr_tfidf = Pipeline([("vect", tfidf), ("clf", LogisticRegression(solver="saga"))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=2, n_jobs=1)

gs_lr_tfidf.fit(X_train, y_train)





Fitting 5 folds for each of 32 candidates, totalling 160 fits


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=0.1, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7fb15b695510>; total time=   3.2s


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=0.1, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7fb15b695510>; total time=   5.9s


...

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 2), vect__norm=None, vect__stop_words=["it's", 're', 'doesn', 'couldn', 'weren', 'same', 'from', 'the', 'she', 'this', 'SouthwestAir', "shan't", 'being', 'we', 'you', 'each', 'or', 'most', 's', 'few', 'through', 'in', 'on', 'do', 'shouldn', 'your', 'our', 'shan', "you're", 'too', 'd', 'such', 'as', "doesn't", 'off', 'wouldn', 'if', 'when', 'here', "couldn't", 'about', 'mightn', 'with', "wasn't", 'haven', 'does', 'its', 'not', 'only', 'these', "wouldn't", 'why', 'his', 'JetBlue', 'so', 'some', 'below', 'no', "hasn't", 'ourselves', 'yourselves', "that'll", 'for', 'was', 'mustn', 'into', 'who', 'above', 'hasn', 'which', 'an', "you'll", 'y', 've', 'they', 'further', 'all', 'more', 'ain', 'them', 'are', 'doing', 'yours', 'where', "mightn't", 'now', 'because', 'between', 'until', "should've", 'that', 'myself', 'to', 'again', 'once', 'other', "shouldn't", "mustn't", 'their', 't', 'wasn', "haven't", 'yourself', 'have', 'did', 'there',

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 2), vect__norm=None, vect__stop_words=["it's", 're', 'doesn', 'couldn', 'weren', 'same', 'from', 'the', 'she', 'this', 'SouthwestAir', "shan't", 'being', 'we', 'you', 'each', 'or', 'most', 's', 'few', 'through', 'in', 'on', 'do', 'shouldn', 'your', 'our', 'shan', "you're", 'too', 'd', 'such', 'as', "doesn't", 'off', 'wouldn', 'if', 'when', 'here', "couldn't", 'about', 'mightn', 'with', "wasn't", 'haven', 'does', 'its', 'not', 'only', 'these', "wouldn't", 'why', 'his', 'JetBlue', 'so', 'some', 'below', 'no', "hasn't", 'ourselves', 'yourselves', "that'll", 'for', 'was', 'mustn', 'into', 'who', 'above', 'hasn', 'which', 'an', "you'll", 'y', 've', 'they', 'further', 'all', 'more', 'ain', 'them', 'are', 'doing', 'yours', 'where', "mightn't", 'now', 'because', 'between', 'until', "should've", 'that', 'myself', 'to', 'again', 'once', 'other', "shouldn't", "mustn't", 'their', 't', 'wasn', "haven't", 'yourself', 'have', 'did', 'there',

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


In [ ]:
print(gs_lr_tfidf.best_params_)
print(gs_lr_tfidf.best_score_)

In [ ]:
y_predicted = gs_lr_tfidf.best_estimator_.predict(X_test)

In [ ]:
gs_lr_tfidf.best_estimator_.score(X_test, y_test)

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS

# # Build the vectorizer
# vect = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS, ngram_range=(1, 2), max_features=200,
#                     token_pattern=r'\b[^\d\W][^\d\W]+\b').fit(tweets.text)
# # Create sparse matrix from the vectorizer
# X_tfidf = vect.transform(tweets.text)

# # Create a DataFrame
# X_tfidf_transformed = pd.DataFrame(X_tfidf.toarray(), columns=vect.get_feature_names_out())
# X_tfidf_transformed.head()
# # can also do this: max_df= 0.9, min_df=0.01

In [ ]:
tweets

In [ ]:
data = pd.concat([ X_tfidf_transformed, tweets], axis=1)

In [ ]:
data

In [ ]:
# Lastly, build the classifier


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Train a logistic regression
log_reg = LogisticRegression(solver="newton-cg", C= 1).fit(X_train, y_train)

# Predict the labels
y_predicted = log_reg.predict(X_test)

# Print accuracy score and confusion matrix on test set
print('Accuracy on the test set: ', accuracy_score(y_test, y_predicted))
# print(confusion_matrix(y_test, y_predicted)/len(y_test))
print(confusion_matrix(y_test, y_predicted))